<a href="https://colab.research.google.com/github/sudeshna0501/stock-market-analysis/blob/main/Stock_Market_Analysis_Atlys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importing necessary libraries

import requests
import pandas as pd
from datetime import datetime, timedelta
import sqlite3
from statistics import median


In [2]:
#Declaring the API KEY
#We can put this in a secure key vault

API_KEY = ~


In [3]:
#Declaring the start and end date to fetch historical data dump

start_date = '2020-01-01'
end_date = '2024-05-31'


In [4]:
#List of Top 10 companies in India (by Market Cap) - Self declared for usage in url

companies = ['RELIANCE', 'TCS', 'HDFCBANK', 'HINDUNILVR', 'INFY', 'HDFC', 'ICICIBANK', 'KOTAKBANK', 'BHARTIARTL', 'HCLTECH']


Script for historical dump

In [5]:
# Function to fetch the data by necessary format

def stock_data(symbol):
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}.BO&outputsize=full&apikey={API_KEY}"
    r = requests.get(url)
    data = r.json()

    if 'Time Series (Daily)' in data:
        stock = pd.DataFrame(data['Time Series (Daily)']).transpose()
        stock.index = pd.to_datetime(stock.index)
        stock.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
        stock['Company'] = symbol
        stock = stock[['Company', 'Open', 'High', 'Low', 'Close', 'Volume']]
        return stock
    else:
        return pd.DataFrame()


In [7]:
# Fetching data per company

per_company = []
for company in companies:
    company_data = stock_data(company)
    if not company_data.empty:
        per_company.append(company_data)


#Concatenate all companies data

if per_company:
    total_data = pd.concat(per_company)
    total_data.reset_index(inplace=True)
    total_data.rename(columns={'index': 'Date'}, inplace=True)
    total_data = total_data[['Date', 'Company', 'Open', 'High', 'Low', 'Close', 'Volume']]
    total_data.to_csv('total_data.csv', index=False)
else:
    print("There is no data. Try again!")


In [8]:
total_data

,Date,Company,Open,High,Low,Close,Volume
0,2024-07-05,RELIANCE,3114.6000,3197.6500,3097.0000,3188.0000,758770
1,2024-07-04,RELIANCE,3116.9500,3134.5000,3102.3000,3107.9000,132480
2,2024-07-03,RELIANCE,3130.3500,3149.5000,3085.2000,3105.3000,171680
3,2024-07-02,RELIANCE,3133.8000,3150.0000,3113.3500,3132.3000,157110
4,2024-07-01,RELIANCE,3129.9500,3157.5500,3110.4000,3120.3500,187610
...,...,...,...,...,...,...,...
43288,2005-01-07,HCLTECH,329.9500,329.9500,321.9500,325.4000,220256
43289,2005-01-06,HCLTECH,333.0000,334.0000,319.0000,322.0000,908712
43290,2005-01-05,HCLTECH,348.1000,349.5000,327.1000,334.0500,237496
43291,2005-01-04,HCLTECH,351.0000,352.3000,345.0500,347.8000,339584


In [9]:
# Fetching historical data from Jan 1, 2020 to May 31, 2024

historical_data = total_data[(total_data['Date'] >= start_date) & (total_data['Date'] <= end_date)]
historical_data


,Date,Company,Open,High,Low,Close,Volume
24,2024-05-31,RELIANCE,2864.6500,2884.2000,2843.2500,2859.6000,797290
25,2024-05-30,RELIANCE,2884.0000,2884.0000,2841.5000,2850.0000,54130
26,2024-05-29,RELIANCE,2897.0000,2916.1000,2877.0000,2881.4500,65080
27,2024-05-28,RELIANCE,2941.9500,2957.0000,2905.4000,2911.2500,116320
28,2024-05-27,RELIANCE,2960.8000,2967.4500,2924.0000,2932.4500,134570
...,...,...,...,...,...,...,...
39598,2020-01-07,HCLTECH,585.0000,588.0000,579.3000,584.2500,32259
39599,2020-01-06,HCLTECH,584.1000,591.9000,579.8000,581.9000,155603
39600,2020-01-03,HCLTECH,575.0000,588.5000,573.2500,584.6500,85559
39601,2020-01-02,HCLTECH,575.0000,575.0000,570.0500,573.4500,30454


Script for daily dump (D-1)

In [ ]:
#We check the date for which day the data is available. For example, while running the query on 08-07-2024, data was available only till 4th, hence yesterday's date will give no data in the subsequent query
total_data[total_data['Date'] == '2024-07-04']

In [16]:
dt_1 = datetime.now()-timedelta(days=1)
dt_2 = dt_1.strftime('%Y-%m-%d')
yesterday_data = total_data[total_data['Date'].dt.date == dt_1.date()]

,Date,Company,Open,High,Low,Close,Volume


SQL DB/DW credentials such that querying can be done - Connection to SQLITE3

In [28]:
# Connect to SQLite databas
conn = sqlite3.connect('historical_data.db')

# Cursor object to interact with the database
cursor = conn.cursor()

#Converting dataframe to SQL table
historical_data.to_sql('historical_data_table', conn, if_exists='replace', index=False)

# Indexing of the DB on Company and Date
cursor.execute("CREATE INDEX idx_company ON historical_data_table (Company)")
cursor.execute("CREATE INDEX idx_date ON historical_data_table (Date)")


# Company Wise Daily Variation of Prices
cursor.execute("""
        SELECT Date, Company, (Close - Open) AS DailyVariation
        FROM historical_data_table
        WHERE Company = 'HCLTECH'
        ORDER BY Date DESC
    """)
data1 = cursor.fetchall()

# Company Wise Daily Volume Change
cursor.execute("""
        SELECT Date, Company, Volume
        FROM historical_data_table
        WHERE Company = 'TCS'
        ORDER BY Date DESC;
    """)
data2 = cursor.fetchall()


# Median Daily Variation - Unable to work this out
# cursor.execute("""
#         SELECT Median(Close - Open) AS DailyVariation
#         FROM historical_data_table
#         WHERE Company = 'ICICIBANK'
#     """)
data3 = cursor.fetchall()

for data in data2: #data1 to be changed to data2 and data3, depending on the result desired to be obtained
  print(data)

conn.commit()
conn.close()

('2024-05-31 00:00:00', 'TCS', '350824')
('2024-05-30 00:00:00', 'TCS', '169744')
('2024-05-29 00:00:00', 'TCS', '93331')
('2024-05-28 00:00:00', 'TCS', '27095')
('2024-05-27 00:00:00', 'TCS', '99435')
('2024-05-24 00:00:00', 'TCS', '163855')
('2024-05-23 00:00:00', 'TCS', '39379')
('2024-05-22 00:00:00', 'TCS', '88374')
('2024-05-21 00:00:00', 'TCS', '90317')
('2024-05-18 00:00:00', 'TCS', '4203')
('2024-05-17 00:00:00', 'TCS', '223617')
('2024-05-16 00:00:00', 'TCS', '33443')
('2024-05-15 00:00:00', 'TCS', '71057')
('2024-05-14 00:00:00', 'TCS', '19294')
('2024-05-13 00:00:00', 'TCS', '68499')
('2024-05-10 00:00:00', 'TCS', '119844')
('2024-05-09 00:00:00', 'TCS', '24876')
('2024-05-08 00:00:00', 'TCS', '25099')
('2024-05-07 00:00:00', 'TCS', '68881')
('2024-05-06 00:00:00', 'TCS', '53334')
('2024-05-03 00:00:00', 'TCS', '149072')
('2024-05-02 00:00:00', 'TCS', '76798')
('2024-04-30 00:00:00', 'TCS', '98711')
('2024-04-29 00:00:00', 'TCS', '22303')
('2024-04-26 00:00:00', 'TCS', '114